# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [149]:
import main
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

In [150]:
def f1(length):
    zad1 = pd.read_sql(f"select title, length from film where length = {length}", con=connection)
    return zad1
f1(180)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,length
0,Alley Evolution,180
1,Confidential Interview,180
2,Impact Aladdin,180
3,Mixed Doors,180
4,Mussolini Spoilers,180
5,Nash Chocolat,180
6,Something Duck,180


In [151]:
def f2(city):
    zad2 = pd.read_sql(f"""select first_name, last_name, city from customer 
        inner join address on customer.address_id = address.address_id 
        inner join city on address.city_id = city.city_id 
        where city = '{city}'
        """, con=connection)
    return zad2
f2('Adana')

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,city
0,Larry,Thrasher,Adana


In [152]:
def f3():
    zad3 = pd.read_sql("select avg(amount) from payment", con=connection)
    return zad3
f3()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg
0,4.200606


In [153]:
def f4():
    zad4 = pd.read_sql("""select count(distinct film_id), name from category 
        inner join film_category on category.category_id = film_category.category_id
        group by category.category_id
        """, con=connection)
    return zad4
f4()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count,name
0,64,Action
1,66,Animation
2,60,Children
3,57,Classics
4,58,Comedy
5,68,Documentary
6,62,Drama
7,69,Family
8,73,Foreign
9,61,Games


In [154]:
def f5():
    zad5 = pd.read_sql("""select country, count(distinct customer_id) from customer
        inner join address on customer.address_id = address.address_id
        inner join city on address.city_id = city.city_id
        inner join country on city.country_id = country.country_id
        group by country
    """, con=connection)
    return zad5
f5()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


In [155]:
# def f6():
#     zad6 = pd.read_sql("""select count(distinct customer_id), store.store_id from store
#         inner join customer on store.store_id = customer.store_id
#         inner join staff on store.store_id = staff.store_id
#         group by store.store_id
#         having count(distinct customer_id) between 100 and 300
#     """, con=connection)
#     return zad6
# f6()

def f6():
    zad6 = pd.read_sql("""
        select store.store_id, address, city, country from store
        inner join staff on store.store_id = staff.store_id
        inner join address on staff.address_id = address.address_id
        inner join city on address.city_id = city.city_id
        inner join country on city.country_id = country.country_id
        where store.store_id in (
            select store.store_id from store
            inner join customer on store.store_id = customer.store_id
            inner join staff on store.store_id = staff.store_id
            inner join address on staff.address_id = address.address_id
            group by store.store_id
            having count(distinct customer_id) between 100 and 300)
    """, con=connection)
    return zad6
f6()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,store_id,address,city,country
0,2,1411 Lillydale Drive,Woodridge,Australia


In [156]:
def f7():
    zad7 = pd.read_sql("""select first_name, last_name, sum(length) from film
        inner join inventory on film.film_id = inventory.film_id
        inner join rental on inventory.inventory_id = rental.inventory_id
        inner join customer on rental.customer_id = customer.customer_id
        group by first_name, last_name
        having sum(length) > 200 * 60
    """, con=connection)
    return zad7
f7()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,sum


In [157]:
def f8():
    zad8 = pd.read_sql("""select avg(rental_duration) from film
    """, con=connection)
    return zad8
f8()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg
0,4.985


In [158]:
def f9():
    zad9 = pd.read_sql("""select name, avg(length) from film
        inner join film_category on film.film_id = film_category.film_id
        inner join category on film_category.category_id = category.category_id
        group by name
    """, con=connection)
    return zad9
f9()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


In [159]:
# def f10():
#     zad10 = pd.read_sql("""select name, max(length(title)) from film
#         inner join film_category on film.film_id = film_category.film_id
#         inner join category on film_category.category_id = category.category_id
#         group by name
#     """, con=connection)
#     return zad10
# f10()

def f10():
    zad10 = pd.read_sql("""
        select name, title, length(title) from film
        inner join film_category on film.film_id = film_category.film_id
        inner join category as c1 on film_category.category_id = c1.category_id
        where length(title) = ( select max( length(title) ) 
            from film
            inner join film_category on film.film_id = film_category.film_id
            inner join category as c2 on film_category.category_id = c2.category_id 
            where c1.name = c2.name )
        order by name
    """, con=connection)
    return zad10
f10()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,title,length
0,Action,Entrapment Satisfaction,23
1,Animation,Telemark Heartbreakers,22
2,Children,Heartbreakers Bright,20
3,Children,Microcosmos Paradise,20
4,Children,Sweethearts Suspects,20
5,Classics,Extraordinary Conquerer,23
6,Comedy,Trainspotting Strangers,23
7,Documentary,Intolerable Intentions,22
8,Documentary,Deliverance Mulholland,22
9,Drama,Goldfinger Sensibility,22


In [160]:
# def f11():
#     zad11 = pd.read_sql("""select name, max(length) from film
#         inner join film_category on film.film_id = film_category.film_id
#         inner join category on film_category.category_id = category.category_id
#         group by name
#     """, con=connection)
#     return zad11
# f11()

def f11():
    zad11 = pd.read_sql("""
        select name, title, length from film
        inner join film_category on film.film_id = film_category.film_id
        inner join category as c1 on film_category.category_id = c1.category_id
        where length = ( select max( film.length ) 
            from film
            inner join film_category on film.film_id = film_category.film_id
            inner join category as c2 on film_category.category_id = c2.category_id 
            where c1.name = c2.name )
        order by name
    """, con=connection)
    return zad11
f11()

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,title,length
0,Action,Darn Forrester,185
1,Action,Worst Banger,185
2,Animation,Pond Seattle,185
3,Animation,Gangs Pride,185
4,Children,Fury Murder,178
5,Children,Wrong Behavior,178
6,Classics,Conspiracy Spirit,184
7,Comedy,Control Anthem,185
8,Documentary,Young Language,183
9,Documentary,Wife Turn,183


In [161]:
import main
main.film_in_category(1)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


In [162]:
main.number_films_in_category(1)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,count
0,Action,64


In [163]:
main.number_film_by_length(50, 100)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,length,count
0,50,9
1,51,7
2,52,7
3,53,9
4,54,6
5,55,2
6,56,5
7,57,7
8,58,7
9,59,9


In [164]:
main.client_from_city('Athenai')

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city,first_name,last_name
0,Athenai,Linda,Williams


In [165]:
main.avg_amount_by_length(48)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,length,avg
0,48,4.295389


In [166]:
main.client_by_sum_length(1000)

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Clara,Shaw,4808
595,Wesley,Bull,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [167]:
main.category_statistic_length('Action')

c:\Users\tomis\anaconda3\envs\tomislaw_tarnawski\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
